<a href="https://colab.research.google.com/github/Namyalg/Topic-Modeling/blob/master/COVID_Title_Khushboo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("punkt")
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
stop = set(stopwords.words('english'))
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
import string
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
import csv
from csv import reader
tit = []
with open("/content/drive/My Drive/topic modelling dataset/COVID-19_title.csv", 'r') as read_obj:
    # pass the file object to reader() to get the reader object
    csv_reader = reader(read_obj)
    # Iterate over each row in the csv using reader object
    for row in csv_reader:
        # row variable is a list that represents a row in csv
        tit.append(row)

In [5]:
val = []
for i in tit:
  val.append(i[0])

Using Bag of Words

In [13]:
tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = set(stopwords.words('english'))
# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
    
doc_set = val
# list for tokenized documents in loop
texts = []

# loop through document list
for i in doc_set:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)
    texts.append(tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
    
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]
corpus[:2]


[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1)],
 [(14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1)]]

In [14]:
preview = corpus[-1]
for i in range(len(preview)):
    print("Word {} (\"{}\") appears {} time.".format(preview[i][0], 
                                               dictionary[preview[i][0]], preview[i][1]))

Word 167 ("genomic") appears 1 time.
Word 268 ("2") appears 1 time.
Word 389 ("approach") appears 1 time.
Word 551 ("computational") appears 1 time.
Word 698 ("biology") appears 1 time.
Word 1080 ("variations") appears 1 time.
Word 1201 ("cov") appears 1 time.
Word 1202 ("sars") appears 1 time.
Word 1290 ("glycoprotein") appears 1 time.
Word 2058 ("proteomic") appears 1 time.
Word 2435 ("spike") appears 1 time.
Word 4902 ("exploring") appears 1 time.


Using TF-IDF

In [15]:
from gensim import corpora, models
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.4008391105373045),
 (1, 0.24567875372913753),
 (2, 0.12458921372558343),
 (3, 0.25496299665686567),
 (4, 0.2004734464233877),
 (5, 0.16736632379793243),
 (6, 0.16468704915717103),
 (7, 0.3668081059363934),
 (8, 0.3197397997903073),
 (9, 0.2707589909539692),
 (10, 0.2722632346863148),
 (11, 0.3321192241618655),
 (12, 0.23860630098681238),
 (13, 0.2272946768906055)]


Running LDA using Bag of Words

In [16]:
lda_model = gensim.models.LdaMulticore(corpus, num_topics=8, id2word=dictionary, passes=2, workers=2)

In [17]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.172*"covid" + 0.168*"19" + 0.022*"coronavirus" + 0.019*"pandem" + 0.018*"disease" + 0.018*"pandemic" + 0.017*"2019" + 0.015*"patients" + 0.008*"during" + 0.007*"clinical"
Topic: 1 
Words: 0.050*"sars" + 0.046*"cov" + 0.043*"2" + 0.021*"virus" + 0.018*"coronavirus" + 0.015*"human" + 0.014*"protein" + 0.012*"viral" + 0.010*"detection" + 0.010*"infection"
Topic: 2 
Words: 0.010*"angiotensin" + 0.009*"induced" + 0.008*"system" + 0.008*"lung" + 0.006*"role" + 0.006*"enzyme" + 0.005*"1" + 0.005*"receptor" + 0.005*"inhibitors" + 0.004*"cell"
Topic: 3 
Words: 0.032*"health" + 0.011*"pandemic" + 0.011*"care" + 0.010*"review" + 0.009*"public" + 0.008*"among" + 0.008*"outbreak" + 0.007*"social" + 0.007*"global" + 0.007*"healthcare"
Topic: 4 
Words: 0.014*"tract" + 0.011*"protective" + 0.009*"personal" + 0.009*"infections" + 0.007*"asthma" + 0.007*"policy" + 0.006*"telemedicine" + 0.006*"equipment" + 0.005*"chloroquine" + 0.005*"viral"
Topic: 5 
Words: 0.037*"virus" + 0.019*"inf

Running LDA using TF-IDF

In [19]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=8, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))
    print("\n")

Topic: 0 Word: 0.010*"covid" + 0.009*"19" + 0.005*"pandem" + 0.004*"pandemic" + 0.003*"hydroxychloroquine" + 0.003*"stroke" + 0.003*"chloroquine" + 0.002*"during" + 0.002*"reflections" + 0.002*"coronavirus"


Topic: 1 Word: 0.007*"covid" + 0.007*"19" + 0.004*"anxiety" + 0.003*"fatality" + 0.003*"storm" + 0.003*"pandem" + 0.002*"pandemic" + 0.002*"radiology" + 0.002*"caring" + 0.002*"embolism"


Topic: 2 Word: 0.009*"covid" + 0.009*"19" + 0.004*"social" + 0.004*"pandem" + 0.004*"letter" + 0.004*"distancing" + 0.004*"pandemic" + 0.004*"health" + 0.004*"editor" + 0.003*"coronavirus"


Topic: 3 Word: 0.009*"19" + 0.009*"covid" + 0.006*"angiotensin" + 0.004*"patients" + 0.004*"disease" + 0.004*"wuhan" + 0.004*"coronavirus" + 0.003*"diabetes" + 0.003*"system" + 0.003*"china"


Topic: 4 Word: 0.012*"virus" + 0.010*"sars" + 0.009*"cov" + 0.008*"2" + 0.007*"coronavirus" + 0.007*"protein" + 0.006*"human" + 0.005*"viral" + 0.004*"respiratory" + 0.004*"porcine"


Topic: 5 Word: 0.011*"respiratory"

Performance evaluation of LDA using Bag of Words

In [20]:
texts[13]

['vaccinia',
 'virus',
 'infection',
 'disrupts',
 'microtubule',
 'organization',
 'centrosome',
 'funct']

In [21]:
for index, score in sorted(lda_model[corpus[13]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 8)))


Score: 0.7270503044128418	 
Topic: 0.050*"sars" + 0.046*"cov" + 0.043*"2" + 0.021*"virus" + 0.018*"coronavirus" + 0.015*"human" + 0.014*"protein" + 0.012*"viral"

Score: 0.18955178558826447	 
Topic: 0.019*"case" + 0.017*"infection" + 0.014*"sars" + 0.009*"transmission" + 0.008*"cov" + 0.008*"report" + 0.007*"model" + 0.007*"2"

Score: 0.013915383256971836	 
Topic: 0.032*"health" + 0.011*"pandemic" + 0.011*"care" + 0.010*"review" + 0.009*"public" + 0.008*"among" + 0.008*"outbreak" + 0.007*"social"

Score: 0.01390310749411583	 
Topic: 0.010*"angiotensin" + 0.009*"induced" + 0.008*"system" + 0.008*"lung" + 0.006*"role" + 0.006*"enzyme" + 0.005*"1" + 0.005*"receptor"

Score: 0.013900857418775558	 
Topic: 0.037*"virus" + 0.019*"infectious" + 0.018*"influenza" + 0.013*"porcine" + 0.008*"chapter" + 0.008*"emerging" + 0.008*"coronavirus" + 0.007*"diarrhea"

Score: 0.013894373551011086	 
Topic: 0.014*"tract" + 0.011*"protective" + 0.009*"personal" + 0.009*"infections" + 0.007*"asthma" + 0.007*

Performance evaluation of LDA using TF-IDF 

In [23]:
for index, score in sorted(lda_model_tfidf[corpus[13]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 8)))


Score: 0.5284973382949829	 
Topic: 0.012*"virus" + 0.010*"sars" + 0.009*"cov" + 0.008*"2" + 0.007*"coronavirus" + 0.007*"protein" + 0.006*"human" + 0.005*"viral"

Score: 0.3881185054779053	 
Topic: 0.004*"laparoscopic" + 0.004*"covid" + 0.004*"19" + 0.004*"plasma" + 0.004*"convalescent" + 0.004*"surgery" + 0.003*"single" + 0.003*"intubation"

Score: 0.013904414139688015	 
Topic: 0.014*"covid" + 0.014*"19" + 0.010*"health" + 0.009*"pandemic" + 0.008*"pandem" + 0.007*"care" + 0.006*"outbreak" + 0.005*"impact"

Score: 0.013899390585720539	 
Topic: 0.009*"19" + 0.009*"covid" + 0.006*"angiotensin" + 0.004*"patients" + 0.004*"disease" + 0.004*"wuhan" + 0.004*"coronavirus" + 0.003*"diabetes"

Score: 0.013899218291044235	 
Topic: 0.011*"respiratory" + 0.011*"patients" + 0.011*"19" + 0.011*"covid" + 0.010*"coronavirus" + 0.009*"2019" + 0.009*"disease" + 0.008*"acute"

Score: 0.013897056691348553	 
Topic: 0.009*"covid" + 0.009*"19" + 0.004*"social" + 0.004*"pandem" + 0.004*"letter" + 0.004*"dis